In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import spacy

In [2]:
df = pd.read_csv("diario_emociones.csv")

# Balancear clases por sobremuestreo
dfs = []
for emocion in df["emocion"].unique():
    subset = df[df["emocion"] == emocion]
    dfs.append(resample(subset, 
                      replace=True, 
                      n_samples=500,  # Ajustar según necesidad
                      random_state=42))
df = pd.concat(dfs, ignore_index=True)

In [3]:
nlp = spacy.load("es_core_news_sm")

def limpiar_texto(texto):
    doc = nlp(texto.lower())
    tokens = [
        token.lemma_ 
        for token in doc 
        if not token.is_stop 
        and not token.is_punct
        and token.is_alpha
    ]
    return " ".join(tokens)

df["texto_limpio"] = df["texto"].apply(limpiar_texto)

In [4]:
vectorizer = TfidfVectorizer(
    max_features=3000,
    ngram_range=(1, 2),  # Bigramas
    min_df=5,            # Ignorar términos raros
    max_df=0.9          # Ignorar términos muy comunes
)
X = vectorizer.fit_transform(df["texto_limpio"]).toarray()

# Codificación de emociones
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df["emocion"])



In [5]:
model = Sequential()
model.add(Dense(
    256, 
    input_shape=(X.shape[1],),
    activation='relu',
    kernel_regularizer=l2(0.02),
    activity_regularizer=l2(0.01)
))
model.add(Dropout(0.7))
model.add(Dense(
    128,
    activation='relu',
    kernel_regularizer=l2(0.01))
)
model.add(Dropout(0.6))
model.add(Dense(
    len(label_encoder.classes_), 
    activation='softmax'
))

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


C:\Users\walte\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    batch_size=64,
    epochs=100,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.2634 - loss: 8.1040 - val_accuracy: 0.8140 - val_loss: 4.5293
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3901 - loss: 3.8867 - val_accuracy: 0.7640 - val_loss: 2.5254
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4789 - loss: 2.3045 - val_accuracy: 0.9060 - val_loss: 1.8481
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6715 - loss: 1.7677 - val_accuracy: 1.0000 - val_loss: 1.5650
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7448 - loss: 1.5237 - val_accuracy: 1.0000 - val_loss: 1.2856
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8980 - loss: 1.2701 - val_accuracy: 1.0000 - val_loss: 1.0089
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9525 - loss: 1.0524 - val_accuracy: 1.0000 - val_loss: 0.8245
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9774 - loss: 0.9222 - val_accuracy: 1.0000 - 

In [7]:
def predecir_emocion(texto, umbral_confianza=0.65):
    texto_limpio = limpiar_texto(texto)
    texto_vector = vectorizer.transform([texto_limpio]).toarray()
    prediccion = model.predict(texto_vector, verbose=0)
    
    if np.max(prediccion) < umbral_confianza:
        return "indefinido", np.max(prediccion)
    
    return label_encoder.inverse_transform([np.argmax(prediccion)])[0], np.max(prediccion)

In [8]:
def chatbot_respuesta(texto):
    emocion, confianza = predecir_emocion(texto)
    
    recursos = {
        "tristeza": {
            "mensaje": "🔵 Pareces sentir tristeza. Te recomiendo:",
            "acciones": [
                "Meditación guiada 'Alivio emocional' (10 min)",
                "Ejercicio: Escribe una carta para liberar emociones"
            ]
        },
        "ansiedad": {
            "mensaje": "🟠 Detecto señales de ansiedad. Prueba:",
            "acciones": [
                "Técnica de respiración 4-7-8 (instrucciones)",
                "Ejercicio de grounding: 5-4-3-2-1"
            ]
        },
        "estrés": {
            "mensaje": "🟢 Sugerencias para manejar el estrés:",
            "acciones": [
                "Meditación 'Libera tensiones' (15 min)",
                "Prioriza tareas con matriz Eisenhower"
            ]
        },
        "enojo": {
            "mensaje": "🔴 Detecto frustración. Intenta:",
            "acciones": [
                "Ejercicio físico de alta intensidad",
                "Técnica de pausa consciente de 5 minutos"
            ]
        },
        "felicidad": {
            "mensaje": "🟡 ¡Me alegra verte así! Mantén esto:",
            "acciones": [
                "Registra este momento en tu diario positivo",
                "Comparte tu estado con alguien especial"
            ]
        },
        "indefinido": {
            "mensaje": "⚪️ Necesito entenderte mejor. ¿Podrías:",
            "acciones": [
                "Describir cómo te sientes con más detalle?",
                "Contarme qué ha pasado recientemente?"
            ]
        }
    }
    
    respuesta = recursos.get(emocion, recursos["indefinido"])
    return (
        f"{respuesta['mensaje']}\n" + 
        "\n".join([f"- {accion}" for accion in respuesta['acciones']]) + 
        f"\n\nConfianza del modelo: {confianza:.2%}"
    )


In [9]:
test_cases = [
    "No tengo ganas de salir de la cama hoy",
    "Estoy harto de las reuniones inútiles",
    "Me palpita el corazón muy rápido",
    "Logré terminar todos mis pendientes a tiempo!",
    "El tráfico me hizo llegar tarde otra vez"
]

for caso in test_cases:
    print(f"🧑 Usuario: {caso}")
    print(f"🤖 Chatbot:\n{chatbot_respuesta(caso)}\n")
    print("―" * 50)

🧑 Usuario: No tengo ganas de salir de la cama hoy
🤖 Chatbot:
🟠 Detecto señales de ansiedad. Prueba:
- Técnica de respiración 4-7-8 (instrucciones)
- Ejercicio de grounding: 5-4-3-2-1

Confianza del modelo: 70.60%

――――――――――――――――――――――――――――――――――――――――――――――――――
🧑 Usuario: Estoy harto de las reuniones inútiles
🤖 Chatbot:
🔴 Detecto frustración. Intenta:
- Ejercicio físico de alta intensidad
- Técnica de pausa consciente de 5 minutos

Confianza del modelo: 89.11%

――――――――――――――――――――――――――――――――――――――――――――――――――
🧑 Usuario: Me palpita el corazón muy rápido
🤖 Chatbot:
⚪️ Necesito entenderte mejor. ¿Podrías:
- Describir cómo te sientes con más detalle?
- Contarme qué ha pasado recientemente?

Confianza del modelo: 29.02%

――――――――――――――――――――――――――――――――――――――――――――――――――
🧑 Usuario: Logré terminar todos mis pendientes a tiempo!
🤖 Chatbot:
⚪️ Necesito entenderte mejor. ¿Podrías:
- Describir cómo te sientes con más detalle?
- Contarme qué ha pasado recientemente?

Confianza del modelo: 53